In [9]:
import pandas as pd
import configparser
import openai

# Read credentials
parser = configparser.ConfigParser()
parser.read('./config/config.ini')

# Read raw dataset
data = pd.read_csv("./dataset/processed-data.csv")
data_df = pd.DataFrame(data)

data_df

,prompt,completion
0,hi there,0
1,hey,0
2,hello,0
3,good morning,0
4,good afternoon,0
...,...,...
448,aap kaun ho,6
449,kaise use karein,6
450,use karna sikhao,6
451,chalate kaise hein,6


In [23]:
# Convent CSV to JSONL
data_df.to_json("./dataset/training-data.jsonl", orient='records', lines=True)

In [5]:
# Add OpenAI credentials
openai.api_key = parser['openai']['ACCESS_KEY']

In [4]:
# Lists the fine tune odels created
openai.FineTune.list()

<OpenAIObject list at 0x7f931f1bbd80> JSON: {
  "data": [
    {
      "created_at": 1681950512,
      "fine_tuned_model": "ada:ft-personal-2023-04-20-00-40-20",
      "hyperparams": {
        "batch_size": 1,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-v4aN7upK0Qlkhk8KATkMQmt4",
      "model": "ada",
      "object": "fine-tune",
      "organization_id": "org-RfYZDHQkuAcmLXxzETD5TDdn",
      "result_files": [
        {
          "bytes": 66419,
          "created_at": 1681951222,
          "filename": "compiled_results.csv",
          "id": "file-BgtczzcU1xYH8QkL1uHbojQx",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 21012,
          "created_at": 1681950509,
          "filename": "training-data_prepared_train.jsonl",

In [15]:
# Sublets the dataset into training and validation jsonl
# Formats the dataset for best training results
# Run in terminal

cd <project folder>/packages/classifer/dataset
openai tools fine_tunes.prepare_data -f 'training-data.jsonl' -q

SyntaxError: invalid syntax (3277997256.py, line 5)

In [25]:
# Upload training and validation jsonl file
# Retrieve the ID
# Run this code once to upload the file

# training_data = openai.File.create(
#   file=open("./dataset/training-data_prepared_train.jsonl", "rb"),
#   purpose='fine-tune'
# )
# training_data_file_id = training_data.id

# validation_data = openai.File.create(
#   file=open("./dataset/training-data_prepared_valid.jsonl", "rb"),
#   purpose='fine-tune'
# )
# validation_data_file_id = validation_data.id

In [21]:
# Snippet to delete the file

# openai.File.delete(training_data_file_id)
# openai.File.delete(validation_data_file_id)

<File file id=file-WJtyN7fXLqkittywmYGQTEiW at 0x7f097aa0f510> JSON: {
  "deleted": true,
  "id": "file-WJtyN7fXLqkittywmYGQTEiW",
  "object": "file"
}

In [32]:
# Equivalent CLI query: openai api fine_tunes.create -t "training-data_prepared_train.jsonl" -v "training-data_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 7

# openai.FineTune.create(
#   training_file=parser['openai']['TRAINING_DATA_FILE_ID'],
#   validation_file=parser['openai']['VALIDATION_DATA_FILE_ID'],
#   model="ada",
#   compute_classification_metrics=True,
#   classification_n_classes=7,
#   suffix="ayra"
# )

<FineTune fine-tune id=ft-4Q83zx8ODgBUCJJNcf7K4n56 at 0x7f097a8fa390> JSON: {
  "created_at": 1682493803,
  "events": [
    {
      "created_at": 1682493803,
      "level": "info",
      "message": "Created fine-tune: ft-4Q83zx8ODgBUCJJNcf7K4n56",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "classification_n_classes": 7,
    "compute_classification_metrics": true,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-4Q83zx8ODgBUCJJNcf7K4n56",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-RfYZDHQkuAcmLXxzETD5TDdn",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 21600,
      "created_at": 1682493662,
      "filename": "file",
      "id": "file-MWALlVaPCmGTSd9joUcYWVzh",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updat

In [6]:
openai.FineTune.retrieve(id=parser['openai']['FINE_TUNE_ID'])

<FineTune fine-tune id=ft-4Q83zx8ODgBUCJJNcf7K4n56 at 0x7f931f1da5c0> JSON: {
  "created_at": 1682493803,
  "events": [
    {
      "created_at": 1682493803,
      "level": "info",
      "message": "Created fine-tune: ft-4Q83zx8ODgBUCJJNcf7K4n56",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1682493810,
      "level": "info",
      "message": "Fine-tune costs $0.00",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1682493811,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1682493813,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1682493882,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1682493937,
      "level": "info",
      "message": "Completed epoch 2/4",
      "object": "fine-tune